# Save user private key in GCP KMS

#### Step 1. Activate Cloud Key Management Service (KMS) API
https://console.cloud.google.com/marketplace/product/google/cloudkms.googleapis.com

#### Step 2. Create a Google service account that has the role of Cloud KMS Admin
https://console.cloud.google.com/iam-admin/serviceaccounts

Generate and download a private key for the service account in JSON format

#### Step 3. Install KMS client lib

pip install google-cloud-kms firebase-admin

#### Step 4. Connect to KMS

In [1]:
from google.cloud import kms
from google.oauth2 import service_account

In [2]:
# SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'
SERVICE_ACCOUNT_FILE = '../kms-admin.json'

project_id = 'wallet-login-45c1c'

In [3]:

# Create the credentials object from the service account file.
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# Create the client with the credentials.
client = kms.KeyManagementServiceClient(credentials=credentials)


create a key ring, a key, and a key version. You can specify the name of the key when you create it. 
For example, you can use the following code to create a symmetric encryption key named gladius-key in a key ring named gladius-key-ring in the global location

In [ ]:


# Build the parent name from the project and location.
parent = f"projects/{project_id}/locations/global"

# Build the key ring name.
key_ring_name = f"{parent}/keyRings/gladius-key-ring"

# Build the key name.
key_name = f"{key_ring_name}/cryptoKeys/gladius-key"

# Create the key ring.
key_ring = {}
client.create_key_ring(request={"parent": parent, "key_ring_id": "gladius-key-ring", "key_ring": key_ring})

# Create the key.
purpose = kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT
crypto_key = {"purpose": purpose}
client.create_crypto_key(request={"parent": key_ring_name, "crypto_key_id": "gladius-key", "crypto_key": crypto_key})


### Test KMS

You can use the client library to encrypt and decrypt data with the key you created. For example, you can use the following code to encrypt and decrypt a message using gladius-key

In [4]:

# Build the key name.
key_name = f"projects/{project_id}/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key"

# The message to encrypt.
message = b"Hello Gladius!"

# Encrypt the message.
response = client.encrypt(request={"name": key_name, "plaintext": message})
ciphertext = response.ciphertext

print(ciphertext)

b"\n$\x00\x17ax<Kdts\xccu\xc9\xe3\x8c\xa7\xc8\xd0\xbe'#\x04\xaa\xee\xc7\xd2\xf4U\xe6\x05\xc6\xad\xd2\xec\x1a\xa7Q\x127\x00T\xe6\xeb<\xbf\xc6\n\x9dK\xa0\xa6\xc7\x0e\xa86\x83\xf6\x91$\xb3@?\x1do^\x1a\xf1\xe9p\x1aO\x80x]\xa4\xf2r\xf9]\x82\xf0\x17\x91\x98\x89\xc2\xae1cc\n\xc5r\x0f"


In [5]:
# Decrypt the message.
response = client.decrypt(request={"name": key_name, "ciphertext": ciphertext})
plaintext = response.plaintext

print(plaintext)


b'Hello Gladius!'


### Step 5. Encrypt private_key with Fernet and encryption key stored in Cloud KMS
https://cryptography.io/en/latest/fernet/

In [9]:
# pip install cryptography

from cryptography.fernet import Fernet


Use the generate_key() method of the Fernet class to generate a new fernet key2. The key is a random value, and will be completely different each time you call the method2. The key is a URL-safe base64-encoded 32-byte key2. You need to keep this key safe and secret, as it is the only way to encrypt and decrypt your data with Fernet2. For example:

In [10]:
# this will be our encryption key

key = Fernet.generate_key()

Connect to KMS with SA

In [46]:
from google.cloud import kms
from google.oauth2 import service_account

# Create the credentials object from the service account file.
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# Create the client with the credentials.
client = kms.KeyManagementServiceClient(credentials=credentials)

Create a key ring and a key in Cloud KMS to store your encryption key
https://cloud.google.com/kms/docs/create-encryption-keys

##### Gladius Key Ring - Gladius Key

In [6]:

# Build the parent name from the project and location.
parent = f"projects/{project_id}/locations/global"

# Build the key ring name.
key_ring_name = f"{parent}/keyRings/gladius-key-ring"

# Build the key name.
key_name = f"{key_ring_name}/cryptoKeys/gladius-key"

# Create the key ring.
key_ring = {}
client.create_key_ring(request={"parent": parent, "key_ring_id": "gladius-key-ring", "key_ring": key_ring})

# Create the key.
purpose = kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT
crypto_key = {"purpose": purpose}
client.create_crypto_key(request={"parent": key_ring_name, "crypto_key_id": "gladius-key", "crypto_key": crypto_key})


name: "projects/wallet-login-45c1c/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key"
primary {
  name: "projects/wallet-login-45c1c/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key/cryptoKeyVersions/1"
  state: ENABLED
  protection_level: SOFTWARE
  algorithm: GOOGLE_SYMMETRIC_ENCRYPTION
  create_time {
    seconds: 1687953796
    nanos: 295077766
  }
  generate_time {
    seconds: 1687953796
    nanos: 295077766
  }
}
purpose: ENCRYPT_DECRYPT
create_time {
  seconds: 1687953796
  nanos: 295077766
}
version_template {
  protection_level: SOFTWARE
  algorithm: GOOGLE_SYMMETRIC_ENCRYPTION
}
destroy_scheduled_duration {
  seconds: 86400
}

Use the encrypt() method of the client to encrypt your encryption key with your Cloud KMS key

https://cloud.google.com/kms/docs/create-encryption-keys

In [7]:
key_name

'projects/wallet-login-45c1c/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key'

In [11]:

# Encrypt the encryption key with Google KMS.

response = client.encrypt(request={"name": key_name, "plaintext": key})
encrypted_encryption_key = response.ciphertext


Use the client library to decrypt your encryption key that was encrypted with a Cloud KMS key

https://cloud.google.com/kms/docs/encrypt-decrypt

In [12]:

# Decrypt your encryption key with Google KMS.
response = client.decrypt(request={"name": key_name, "ciphertext": encrypted_encryption_key})
encryption_key = response.plaintext

Create a Fernet object with your encryption key

https://www.geeksforgeeks.org/fernet-symmetric-encryption-using-cryptography-module-in-python/

In [13]:
f = Fernet(encryption_key)


Firebase

In [14]:
import firebase_admin
from firebase_admin import auth, credentials, firestore


if not firebase_admin._apps:
    cred = credentials.Certificate("../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()

# Get a reference to the users collection.
users_ref = db.collection("users")

# Get all documents from the users collection.
users_docs = users_ref.stream()

Use the update() method of the Firestore client to save or update the name field for a specific user document. You need to specify the document reference and a dictionary of fields and values to update. For each name field, use the encrypt() method of Fernet to encrypt it with your encryption key

In [15]:
# Iterate over the documents and encrypt and update the name field.
for user_doc in users_docs:
  # Get the document data as a dictionary.
  user_data = user_doc.to_dict()

  # Get the name field from the data.
  user_name = user_data.get("name")

  # Print the name field.
  #print(user_name)

  # Encrypt the name field with Fernet.
  encrypted_user_name = f.encrypt(user_name.encode())
  #print(encrypted_user_name)

  # Update the document with the encrypted name field.
  user_doc.reference.update({"encrypted_user_name": encrypted_user_name})

